<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 2*
# Convolutional Neural Networks (CNNs)

# Assignment

Load a pretrained network from Keras, [ResNet50](https://tfhub.dev/google/imagenet/resnet_v1_50/classification/1) - a 50 layer deep network trained to recognize [1000 objects](https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt). Starting usage:

```python
import numpy as np

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

ResNet50 = ResNet50(weights='imagenet')
features = model.predict(x)

```

Next you will need to remove the last layer from the ResNet model. Here, we loop over the layers to use the sequential API. There are easier ways to add and remove layers using the Keras functional API, but doing so introduces other complexities. 

```python
# Remote the Last Layer of ResNEt
ResNet50._layers.pop(0)

# Out New Model
model = Sequential()

# Add Pre-trained layers of Old Model to New Model
for layer in ResNet50.layers:
    model.add(layer)

# Turn off additional training of ResNet Layers for speed of assignment
for layer in model.layers:
    layer.trainable = False

# Add New Output Layer to Model
model.add(Dense(1, activation='sigmoid'))
```

Your assignment is to apply the transfer learning above to classify images of Mountains (`./data/mountain/*`) and images of forests (`./data/forest/*`). Treat mountains as the postive class (1) and the forest images as the negative (zero). 

Steps to complete assignment: 
1. Load in Image Data into numpy arrays (`X`) 
2. Create a `y` for the labels
3. Train your model with pretrained layers from resnet
4. Report your model's accuracy

In [4]:
# imports

import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.python.keras import optimizers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.utils import plot_model
from sklearn.model_selection import train_test_split

import zipfile
import pathlib
import random
import os
import matplotlib.pyplot as plt
#import cv2 as cv

In [9]:
# Read in mountain data from file

import os

mtn_img = []

data_files = os.listdir('/Users/ianforrest/Desktop/coding/repos/ianforrest11/DS-Unit-4-Sprint-3-Deep-Learning/module2-convolutional-neural-networks/data/mountain')

for filename in data_files:
    if filename[-3:] == 'jpg':
        path = f'/Users/ianforrest/Desktop/coding/repos/ianforrest11/DS-Unit-4-Sprint-3-Deep-Learning/module2-convolutional-neural-networks/data/mountain/{filename}'
        #print(path)
        with open(path, 'rb') as data:
            content = data.read()
            #print(content)
            mtn_img.append(content)

In [12]:
mtn_img[30][:100]

b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x01\x02\x02\x01\x01\x02\x01\x01\x01\x02\x02\x02\x02\x02\x02\x02\x02\x02\x01\x02\x02\x02\x02\x02\x02\x02\x02\x02\x02\xff\xdb\x00C\x01\x01\x01\x01\x01\x01\x01'

In [13]:
# Read in mountain data from file

import os

for_img = []

data_files = os.listdir('/Users/ianforrest/Desktop/coding/repos/ianforrest11/DS-Unit-4-Sprint-3-Deep-Learning/module2-convolutional-neural-networks/data/forest')

for filename in data_files:
    if filename[-3:] == 'jpg':
        path = f'/Users/ianforrest/Desktop/coding/repos/ianforrest11/DS-Unit-4-Sprint-3-Deep-Learning/module2-convolutional-neural-networks/data/forest/{filename}'
        #print(path)
        with open(path, 'rb') as data:
            content = data.read()
            #print(content)
            for_img.append(content)

In [14]:
for_img[30][:100]

b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x08\x06\x06\x07\x06\x05\x08\x07\x07\x07\t\t\x08\n\x0c\x14\r\x0c\x0b\x0b\x0c\x19\x12\x13\x0f\x14\x1d\x1a\x1f\x1e\x1d\x1a\x1c\x1c $.\' ",#\x1c\x1c(7),01444\x1f\'9=82<.342\xff\xdb\x00C\x01\t\t\t\x0c\x0b\x0c'

In [15]:
# Get the zip files from Google Drive and unzip them into data directory
zip_ref = zipfile.ZipFile("/Users/ianforrest/Desktop/coding/repos/ianforrest11/DS-Unit-4-Sprint-3-Deep-Learning/module2-convolutional-neural-networks/data/forest/forest.zip", 'r')
zip_ref.extractall("data")
zip_ref = zipfile.ZipFile("/Users/ianforrest/Desktop/coding/repos/ianforrest11/DS-Unit-4-Sprint-3-Deep-Learning/module2-convolutional-neural-networks/data/mountain/mountain.zip", 'r')
zip_ref.extractall("data")
zip_ref.close()

data_root = pathlib.Path('data')

In [16]:
# Generate list of image paths. There's a .db file in the image directory
# so only glob all the .jpgs
all_image_paths = list(data_root.glob('*/*.jpg'))
all_image_paths = [str(path) for path in all_image_paths]
random.shuffle(all_image_paths)

image_count = len(all_image_paths)

In [17]:
image_count

702

# Resources and Stretch Goals

Stretch goals
- Enhance your code to use classes/functions and accept terms to search and classes to look for in recognizing the downloaded images (e.g. download images of parties, recognize all that contain balloons)
- Check out [other available pretrained networks](https://tfhub.dev), try some and compare
- Image recognition/classification is somewhat solved, but *relationships* between entities and describing an image is not - check out some of the extended resources (e.g. [Visual Genome](https://visualgenome.org/)) on the topic
- Transfer learning - using images you source yourself, [retrain a classifier](https://www.tensorflow.org/hub/tutorials/image_retraining) with a new category
- (Not CNN related) Use [piexif](https://pypi.org/project/piexif/) to check out the metadata of images passed in to your system - see if they're from a national park! (Note - many images lack GPS metadata, so this won't work in most cases, but still cool)

Resources
- [Deep Residual Learning for Image Recognition](https://arxiv.org/abs/1512.03385) - influential paper (introduced ResNet)
- [YOLO: Real-Time Object Detection](https://pjreddie.com/darknet/yolo/) - an influential convolution based object detection system, focused on inference speed (for applications to e.g. self driving vehicles)
- [R-CNN, Fast R-CNN, Faster R-CNN, YOLO](https://towardsdatascience.com/r-cnn-fast-r-cnn-faster-r-cnn-yolo-object-detection-algorithms-36d53571365e) - comparison of object detection systems
- [Common Objects in Context](http://cocodataset.org/) - a large-scale object detection, segmentation, and captioning dataset
- [Visual Genome](https://visualgenome.org/) - a dataset, a knowledge base, an ongoing effort to connect structured image concepts to language